<a href="https://colab.research.google.com/github/haroldgo23/ph-fake-news-detection/blob/main/Hyperparameter_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [ ]:
import pandas as pd
import numpy as np
import re
import string

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df_eng = pd.read_csv("/content/drive/MyDrive/Machine Learning Tool for Identification of Philippine Fake News in Social Media/English.csv")
df_fil = pd.read_csv("/content/drive/MyDrive/Machine Learning Tool for Identification of Philippine Fake News in Social Media/Filipino.csv")

Mounted at /content/drive


# Preparing the Dataset

In [ ]:
df_merge = pd.concat([df_eng, df_fil], axis =0 )
df = df_merge.drop(["Page"], axis = 1) # combine 
df = df.sample(frac = 1) # shuffle dataset
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)
df.head()

,Text,Label
0,"Rappler, the least-trusted news brand in the P...",1
1,Sa pagpapatuloy ng pagdinig sa Senado ukol sa ...,0
2,Political science teacher named Jan Writer sta...,1
3,Pangungunahan ni Pangulong Ferdinand “Bongbong...,0
4,Nagparating ng pasasalamat si Presidential can...,0


### Define dependent and independent variables

In [ ]:
x = df["Text"]
y = df["Label"]

### Remove extra space etc, lower case

In [ ]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

In [ ]:
df["Text"] = df["Text"].apply(wordopt)  #apply wordopt function

# Train-test split

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# Convert text to vectors

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer()
X = vectorization.fit_transform(x)
xv_train = vectorization.transform(x_train)
xv_test = vectorization.transform(x_test)

# Import Models

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

#Model Hyperparameters

### LR

In [ ]:
param_lr = [{
                'penalty': [None, 'l2'], 
                'C': [0.1, 1, 10, 100, 1000],
                'solver': ['lbfgs', 'newton-cg', 'sag'], 
                'max_iter': [100, 500, 1000, 10000]
            },
            {
                'penalty': ['l1', 'l2'], 
                'C': [0.1, 1, 10, 100, 1000],
                'solver': ['liblinear'], 
                'max_iter': [100, 500, 1000, 10000]
            },
            {
                'penalty': [None, 'l1', 'l2', 'elasticnet'], 
                'C': [0.1, 1, 10, 100, 1000],
                'solver': ['saga'], 
                'max_iter': [100, 500, 1000, 10000]
            }]

### DTC

In [ ]:
param_dtc = {
            'criterion': ['gini', 'entropy', 'log_loss'],
            'max_depth': [None, 2, 3, 4, 5, 6],
            'min_samples_split': [2, 4, 6],
            'max_features': [None, 'auto', 'sqrt', 'log2'],
            'min_samples_leaf': [1, 2]
            }

### RFC

In [ ]:
param_rfc= {
            'n_estimators': [10, 25, 50, 100, 200],
            'criterion': ['gini', 'entropy', 'log_loss'],
            'max_depth': [None, 2, 3, 4, 5, 6],
            'min_samples_split': [2, 4, 6],
            'max_features': [None, 'sqrt', 'log2'],
            'min_samples_leaf': [1, 2]
            }

### GBC

In [ ]:
param_gbc = {
            'learning_rate': [0.01, 0.02, 0.03, 0.04],
            'n_estimators': [100, 500, 1000],
            'subsample': [1.0, 0.8, 0.5, 0.2],
            'max_depth': [3, 4, 6, 8]
             }

### PAC

In [ ]:
param_pac = {
            'C': [0.1, 1, 10, 100, 1000],
            'max_iter': [100, 500, 1000]
             }

### NB

In [ ]:
param_nb =  {
            'alpha': [0]
            #'force_alpha': [True]
             }

### SVM

In [ ]:
param_svm = {
            'C':[0.1, 1, 10, 100, 1000],
            'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            'degree': [0.001, 0.01, 0.1, 1]
             }

# GridSearchCV / RandomizedSearchCV

### LR

In [ ]:
lr = GridSearchCV(LogisticRegression(), param_lr , cv=10, return_train_score=False, verbose=1)
lr = lr.fit(xv_train, y_train)
lr.best_params_

Fitting 10 folds for each of 240 candidates, totalling 2400 fits


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means 

{'C': 1000, 'max_iter': 10000, 'penalty': 'l1', 'solver': 'saga'}

In [ ]:
acc_lr = lr.best_score_
acc_lr

0.9725232919254658

### DTC

In [ ]:
dtc = GridSearchCV(DecisionTreeClassifier(), param_dtc , cv=10, return_train_score=False, verbose=1)
dtc = dtc.fit(xv_train, y_train)
dtc.best_params_

Fitting 10 folds for each of 432 candidates, totalling 4320 fits


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
1440 fits failed out of a total of 4320.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1440 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/tree/_classes.py", line 937, in fit
    super().fit(
  File "/usr/local/lib/python3.8/dist-packages/sklearn/tree/_classes.py", line 352, in fit
    criterion = CRITERIA_CLF[self.criterion](
KeyError: 'log_loss'

  warnings.warn(some_fits_failed_message, Fit

{'criterion': 'entropy',
 'max_depth': None,
 'max_features': None,
 'min_samples_leaf': 1,
 'min_samples_split': 6}

In [ ]:
acc_dtc = dtc.best_score_
acc_dtc

0.9007531055900622

### PAC

In [ ]:
pac = GridSearchCV(PassiveAggressiveClassifier(), param_pac , cv=10, return_train_score=False, verbose=1)
pac = pac.fit(xv_train, y_train)
pac.best_params_

Fitting 10 folds for each of 15 candidates, totalling 150 fits


{'C': 1, 'max_iter': 1000}

In [ ]:
acc_pac = pac.best_score_
acc_pac

0.9737771739130435

### SVM

In [ ]:
svm = GridSearchCV(svm.SVC(probability=True), param_svm , cv=10, return_train_score=False, verbose=1)
svm = svm.fit(xv_train, y_train)
svm.best_params_

Fitting 10 folds for each of 80 candidates, totalling 800 fits


{'C': 1, 'degree': 0.001, 'kernel': 'linear'}

In [ ]:
acc_svm = svm.best_score_
acc_svm

0.9718982919254661

### NB

In [ ]:
nb = GridSearchCV(MultinomialNB(), param_nb , cv=10, return_train_score=False, verbose=1)
nb = nb.fit(xv_train, y_train)
nb.best_params_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


/usr/local/lib/python3.8/dist-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/naive_bayes.py:555: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  warning

{'alpha': 0}

In [ ]:
acc_nb = nb.best_score_
acc_nb

0.9525349378881988

### RFC

In [ ]:
rfc = RandomizedSearchCV(RandomForestClassifier(), param_rfc , cv=10, return_train_score=False, verbose=1, n_iter=100)
rfc = rfc.fit(xv_train, y_train)
rfc.best_params_

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
320 fits failed out of a total of 1000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
320 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/ensemble/_forest.py", line 450, in fit
    trees = Parallel(
  File "/usr/local/lib/python3.8/dist-packages/joblib/parallel.py", line 1085, in __call__
    if self.dispatch_one_batch(iterator):
  File "/usr/local/lib/python3.8/dist-packages/joblib/parallel.py

{'n_estimators': 50,
 'min_samples_split': 6,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': None,
 'criterion': 'entropy'}

In [ ]:
acc_rfc = rfc.best_score_
acc_rfc

0.963179347826087

### GBC

In [ ]:
gbc = RandomizedSearchCV(GradientBoostingClassifier(), param_gbc , cv=10, return_train_score=False, verbose=1, n_iter=10)
gbc = gbc.fit(xv_train, y_train)
gbc.best_params_

Fitting 10 folds for each of 10 candidates, totalling 100 fits


{'subsample': 0.5, 'n_estimators': 1000, 'max_depth': 3, 'learning_rate': 0.02}

In [ ]:
acc_gbc = gbc.best_score_
acc_gbc

0.9687849378881987